In [0]:
# Databricks notebook source
from pyspark.sql import functions as F

spark.conf.set("spark.sql.shuffle.partitions", "8")


In [0]:
silver_tables = {
    "customers": "silver.customers",
    "orders": "silver.orders",
    "order_items": "silver.order_items",
    "products": "silver.products"
}


In [0]:
for name, table in silver_tables.items():
    df = spark.table(table)
    count = df.count()
    print(f"{name}: {count} registros")


In [0]:
required_columns = ["customer_id", "first_name", "last_name"]

df_customers = spark.table("silver.customers")

missing_cols = [c for c in required_columns if c not in df_customers.columns]
assert not missing_cols, f"Colunas faltando: {missing_cols}"


In [0]:
df_customers.groupBy("customer_id") \
    .count() \
    .filter(F.col("count") > 1) \
    .display()


In [0]:
df_orders = spark.table("silver.orders")

invalid_customers = (
    df_orders
    .join(df_customers, "customer_id", "left_anti")
)

invalid_customers.display()
